In [2]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

IMG_SIZE = 256
X_train = [] 
y_train = [] 

train_path = 'C:/Users/VINAY/Desktop/Dataset/Covid19-dataset/train/' 
test_path = 'C:/Users/VINAY/Desktop/Dataset/Covid19-dataset/test/' 

model = None

def read_and_preprocess(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
    img = np.array(img, dtype='float32') 
    img = img/255 
    return img

def get_data(path):
    X = []
    y = []
    for folder in os.scandir(path):
        for entry in os.scandir(path + folder.name):

            X.append(read_and_preprocess(path + folder.name + '/' + entry.name))

            if folder.name[0]=='C':
                y.append(0) 
            elif folder.name[0]=='V':
                y.append(1) 
            else:
                y.append(2) 

    X = np.array(X)
    y = np.array(y)

    return X, y

def image_augmentation():
    global X_train, y_train
    X_aug = []
    y_aug = []

    for i in range(0, len(y_train)):
        X_new = np.fliplr(X_train[i])
        X_aug.append(X_new)
        y_aug.append(y_train[i])

    X_aug = np.array(X_aug)
    y_aug = np.array(y_aug)

    X_train = np.append(X_train, X_aug, axis=0) 
    y_train = np.append(y_train, y_aug, axis=0) 

def split_data(test_size=0.2, random_state=109):
    global X_train, y_train

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, shuffle=True, stratify=y_train, random_state=random_state)
    return X_train, X_val, y_train, y_val

def create_model():
    model = tf.keras.Sequential([
        Conv2D(filters=32, kernel_size=(2,2), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((4,4)),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPooling2D((3,3)),
        Dropout(0.3), 

        Conv2D(filters=64, kernel_size=(4,4), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(5,5), activation='relu', padding='same'),
        MaxPooling2D((2,2)),
        Dropout(0.4),

        Conv2D(filters=128, kernel_size=(5,5), activation='relu', padding='same'),
        MaxPooling2D((2,2)),
        Dropout(0.5),

        Flatten(), 
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')
    ])
    return model

def train_model():
    global X_train, y_train, model

    X_train, X_val, y_train, y_val = split_data(test_size=0.15, random_state=123)

    opt = optimizers.Adam(learning_rate=0.0001)

    model.compile(loss = 'sparse_categorical_crossentropy', optimizer=opt, metrics= ["accuracy"])

    earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

    checkpointer = ModelCheckpoint(filepath="main.h5", verbose=1, save_best_only=True)

    history = model.fit(X_train, y_train, epochs = 100, validation_data=(X_val, y_val), batch_size=32, shuffle=True, callbacks=[earlystopping, checkpointer])

def main():
    global X_train, y_train, model

    X_train, y_train = get_data(train_path)

    image_augmentation()

    model = create_model()

    train_model()

if __name__ == "__main__":
    main()

Epoch 1/100
14/14 [==============================] - ETA: 0s - loss: 1.0870 - accuracy: 0.4390
Epoch 1: val_loss improved from inf to 1.08871, saving model to main.h5
14/14 [==============================] - 11s 682ms/step - loss: 1.0870 - accuracy: 0.4390 - val_loss: 1.0887 - val_accuracy: 0.4474
Epoch 2/100
14/14 [==============================] - ETA: 0s - loss: 1.0891 - accuracy: 0.4507
Epoch 2: val_loss improved from 1.08871 to 1.08548, saving model to main.h5
14/14 [==============================] - 9s 610ms/step - loss: 1.0891 - accuracy: 0.4507 - val_loss: 1.0855 - val_accuracy: 0.4474
Epoch 3/100
14/14 [==============================] - ETA: 0s - loss: 1.0797 - accuracy: 0.4413
Epoch 3: val_loss improved from 1.08548 to 1.07849, saving model to main.h5
14/14 [==============================] - 9s 653ms/step - loss: 1.0797 - accuracy: 0.4413 - val_loss: 1.0785 - val_accuracy: 0.4474
Epoch 4/100
14/14 [==============================] - ETA: 0s - loss: 1.0643 - accuracy: 0.4460
Ep

14/14 [==============================] - ETA: 0s - loss: 0.2750 - accuracy: 0.8897
Epoch 29: val_loss did not improve from 0.15334
14/14 [==============================] - 9s 628ms/step - loss: 0.2750 - accuracy: 0.8897 - val_loss: 0.2168 - val_accuracy: 0.9342
Epoch 30/100
14/14 [==============================] - ETA: 0s - loss: 0.2264 - accuracy: 0.9038
Epoch 30: val_loss did not improve from 0.15334
14/14 [==============================] - 9s 626ms/step - loss: 0.2264 - accuracy: 0.9038 - val_loss: 0.2058 - val_accuracy: 0.9079
Epoch 31/100
14/14 [==============================] - ETA: 0s - loss: 0.2344 - accuracy: 0.9155
Epoch 31: val_loss did not improve from 0.15334
14/14 [==============================] - 9s 627ms/step - loss: 0.2344 - accuracy: 0.9155 - val_loss: 0.1715 - val_accuracy: 0.9342
Epoch 32/100
14/14 [==============================] - ETA: 0s - loss: 0.1869 - accuracy: 0.9296
Epoch 32: val_loss did not improve from 0.15334
14/14 [==============================] - 9s 

Epoch 58/100
14/14 [==============================] - ETA: 0s - loss: 0.1168 - accuracy: 0.9577
Epoch 58: val_loss improved from 0.10375 to 0.08990, saving model to main.h5
14/14 [==============================] - 9s 648ms/step - loss: 0.1168 - accuracy: 0.9577 - val_loss: 0.0899 - val_accuracy: 0.9737
Epoch 59/100
14/14 [==============================] - ETA: 0s - loss: 0.1332 - accuracy: 0.9531
Epoch 59: val_loss did not improve from 0.08990
14/14 [==============================] - 9s 647ms/step - loss: 0.1332 - accuracy: 0.9531 - val_loss: 0.0941 - val_accuracy: 0.9605
Epoch 60/100
14/14 [==============================] - ETA: 0s - loss: 0.1084 - accuracy: 0.9624
Epoch 60: val_loss did not improve from 0.08990
14/14 [==============================] - 10s 711ms/step - loss: 0.1084 - accuracy: 0.9624 - val_loss: 0.1448 - val_accuracy: 0.9474
Epoch 61/100
14/14 [==============================] - ETA: 0s - loss: 0.1416 - accuracy: 0.9413
Epoch 61: val_loss did not improve from 0.08990
1

Epoch 87/100
14/14 [==============================] - ETA: 0s - loss: 0.1006 - accuracy: 0.9671
Epoch 87: val_loss did not improve from 0.03358
14/14 [==============================] - 10s 725ms/step - loss: 0.1006 - accuracy: 0.9671 - val_loss: 0.0583 - val_accuracy: 0.9737
Epoch 88/100
14/14 [==============================] - ETA: 0s - loss: 0.0648 - accuracy: 0.9812
Epoch 88: val_loss did not improve from 0.03358
14/14 [==============================] - 10s 711ms/step - loss: 0.0648 - accuracy: 0.9812 - val_loss: 0.0714 - val_accuracy: 0.9737
Epoch 89/100
14/14 [==============================] - ETA: 0s - loss: 0.0632 - accuracy: 0.9765
Epoch 89: val_loss did not improve from 0.03358
14/14 [==============================] - 10s 744ms/step - loss: 0.0632 - accuracy: 0.9765 - val_loss: 0.0659 - val_accuracy: 0.9868
Epoch 90/100
14/14 [==============================] - ETA: 0s - loss: 0.0533 - accuracy: 0.9789
Epoch 90: val_loss did not improve from 0.03358
14/14 [=====================

NameError: name 'history' is not defined

In [9]:
def save_model():
    global model
    model_json = model.to_json()
    with open("model.json","w") as json_file:
        json_file.write(model_json)

def load_model():
    with open('model.json', 'r') as json_file:
        json_savedModel= json_file.read()

    model = tf.keras.models.model_from_json(json_savedModel)
    model.load_weights('main.h5')
    opt = optimizers.Adam(learning_rate=0.0001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer=opt, metrics= ["accuracy"])
    return model


def evaluate_model(model):

    X_test, y_test = get_data(test_path)

    predictions = model.predict(X_test)

    predict = []

    for i in predictions:
        predict.append(np.argmax(i))

    predict = np.asarray(predict)

    accuracy = accuracy_score(y_test, predict)
    print(accuracy)

    report = classification_report(y_test, predict)
    print(report)

def main():
    global X_train, y_train, model

    X_train, y_train = get_data(train_path)

    save_model()

    model = load_model()

    evaluate_model(model)

if __name__ == "__main__":
    main()


3/3 [==============================] - 0s 77ms/step
0.9545454545454546
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       0.90      0.95      0.93        20
           2       0.95      0.90      0.92        20

    accuracy                           0.95        66
   macro avg       0.95      0.95      0.95        66
weighted avg       0.96      0.95      0.95        66



In [11]:
def convert_to_tflite(model, model_name):
 
    # Convert the model to TFLite format
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Save the TFLite model to disk
    with open(model_name + '.tflite', 'wb') as f:
        f.write(tflite_model)

    print(f'TFLite model saved as {model_name}.tflite')

# Convert the model to TFLite format
convert_to_tflite(model, 'model')

def main():

    model = load_model()

    # Convert the model to TFLite format
    convert_to_tflite(model, 'model')

if __name__ == "__main__":
    main()


INFO:tensorflow:Assets written to: C:\Users\VINAY\AppData\Local\Temp\tmp22jqnnng\assets


INFO:tensorflow:Assets written to: C:\Users\VINAY\AppData\Local\Temp\tmp22jqnnng\assets


TFLite model saved as model.tflite


INFO:tensorflow:Assets written to: C:\Users\VINAY\AppData\Local\Temp\tmp35e01r2y\assets


INFO:tensorflow:Assets written to: C:\Users\VINAY\AppData\Local\Temp\tmp35e01r2y\assets


TFLite model saved as model.tflite
